In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from filters import high_pass_filter, denoise_signal, sinewave_shift, drop_data, count_peaks
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import gc

In [3]:
test_meta = pd.read_csv('data/metadata_test.csv')

In [4]:
test_meta['peaks'] = 0
filtered_dataset = None

test_chunks = [(8712, 8999), (9000, 9999), (10000, 10999), (11000, 11999), 
               (12000, 12999), (13000, 13999), (14000, 14999), (15000, 15999), 
               (16000, 16999), (17000, 17999), (18000, 18999), (19000, 19999), 
               (20000, 20999), (21000, 21999), (22000, 22999), (23000, 23999), 
               (24000, 24999), (25000, 25999), (26000, 26999), (27000, 27999), 
               (28000, 28999), (29000, 29048)]

for chunk_id, chunk in enumerate(test_chunks):
    data_subset = pq.read_pandas('data/test.parquet', 
                                 columns=[str(x) for x in range(chunk[0], chunk[1]+1)]).to_pandas()
    
    for idx, column in enumerate(data_subset.columns):
        new_signal, cutpoint = sinewave_shift(data_subset[column])
        hp_filtered_signal = high_pass_filter(new_signal)
        denoised_sig = denoise_signal(hp_filtered_signal)
        cut_signal = drop_data(denoised_sig)

        test_meta['peaks'].loc[int(column)] = count_peaks(cut_signal)
    
        if isinstance(filtered_dataset, type(None)):
            filtered_dataset = pd.DataFrame(data=cut_signal, columns=[column])
        else:
            filtered_dataset[column] = cut_signal
        
        print("Finished filtering signal Id {}".format(column), end="\r")

    
    filtered_test = pa.Table.from_pandas(filtered_dataset)
    pq.write_table(filtered_test, 'data/filtered_test_{}.parquet'.format(chunk_id))
        
    filtered_dataset = None

In [5]:
test_meta.to_csv('data/filtered_metadata_test.csv', index=False)